In [ ]:
%%writefile quad_tree.cpp
#include <iostream>
#include <vector>
#include <memory>

// Define a Point structure to represent a point in 2D space
struct Point {
    double x;
    double y;
    Point(double x, double y) : x(x), y(y) {}
};

// Define a Rectangle structure to represent a rectangular region
struct Rectangle {
    double xMin, xMax, yMin, yMax;
    Rectangle(double xmin, double xmax, double ymin, double ymax) : xMin(xmin), xMax(xmax), yMin(ymin), yMax(ymax) {}

    // Function to check if a point is contained within the rectangle
    bool contains(const Point& point) const {
        return (point.x >= xMin && point.x <= xMax && point.y >= yMin && point.y <= yMax);
    }
};

// Define the QuadTree Node structure
struct QuadTreeNode {
    Rectangle boundary;
    std::vector<Point> points;
    std::shared_ptr<QuadTreeNode> children[4]; // 0: bottom left, 1: bottom right, 2: top left, 3: top right

    QuadTreeNode(const Rectangle& boundary) : boundary(boundary) {
        for (int i = 0; i < 4; ++i) {
            children[i] = nullptr;
        }
    }
};

// QuadTree class
class QuadTree {
private:
    std::shared_ptr<QuadTreeNode> root;

    void insertHelper(const Point& point, std::shared_ptr<QuadTreeNode>& node, int level) {
        if (!node->boundary.contains(point)) {
            return; // Point does not belong to this node's boundary
        }

        if (node->points.size() < 4) { // Maximum points per node
            node->points.push_back(point);
        } else {
            if (!node->children[0]) { // If children not yet created, create them
                double xMid = (node->boundary.xMin + node->boundary.xMax) / 2;
                double yMid = (node->boundary.yMin + node->boundary.yMax) / 2;

                node->children[0] = std::make_shared<QuadTreeNode>(Rectangle(node->boundary.xMin, xMid, node->boundary.yMin, yMid));
                node->children[1] = std::make_shared<QuadTreeNode>(Rectangle(xMid, node->boundary.xMax, node->boundary.yMin, yMid));
                node->children[2] = std::make_shared<QuadTreeNode>(Rectangle(node->boundary.xMin, xMid, yMid, node->boundary.yMax));
                node->children[3] = std::make_shared<QuadTreeNode>(Rectangle(xMid, node->boundary.xMax, yMid, node->boundary.yMax));
            }

            // Recursively insert into appropriate child node
            for (int i = 0; i < 4; ++i) {
                insertHelper(point, node->children[i], level + 1);
            }
        }
    }

    void printPointsHelper(const std::shared_ptr<QuadTreeNode>& node, int depth) const {
        if (!node) {
            return;
        }

        // Print indentation based on depth
        for (int i = 0; i < depth; ++i) {
            std::cout << "  ";
        }

        std::cout << "Node Boundary: (" << node->boundary.xMin << ", " << node->boundary.yMin << ") - (" << node->boundary.xMax << ", " << node->boundary.yMax << ")\n";
        std::cout << "Points stored in this node:\n";
        for (const auto& point : node->points) {
            std::cout << "(" << point.x << ", " << point.y << ") -z-value: " << depth << "\n";
        }
        std::cout << "\n";

        // Recursively print points stored in child nodes
        for (int i = 0; i < 4; ++i) {
            printPointsHelper(node->children[i], depth + 1);
        }
    }

public:
    QuadTree(const Rectangle& boundary) {
        root = std::make_shared<QuadTreeNode>(boundary);
    }

    void insert(const Point& point) {
        insertHelper(point, root, 0);
    }

    void printPoints() const {
        printPointsHelper(root, 0);
    }
};

int main() {
    Rectangle boundary(0, 100, 0, 100); // Define the boundary of the quad tree
    QuadTree quadTree(boundary);

    // Insert points into the quad tree
    quadTree.insert(Point(10, 10));
    quadTree.insert(Point(20, 20));
    quadTree.insert(Point(30, 30));
    quadTree.insert(Point(74, 43));
    quadTree.insert(Point(90, 90));
    quadTree.insert(Point(5, 5));
    quadTree.insert(Point(15, 15));
    quadTree.insert(Point(35, 35));
    quadTree.insert(Point(54, 25));
    quadTree.insert(Point(60, 60));
    quadTree.insert(Point(70, 70));
    quadTree.insert(Point(80, 80));
      quadTree.insert(Point(40, 6));
    quadTree.insert(Point(4, 70));
    quadTree.insert(Point(2, 80));

    // Print points stored in each node
    quadTree.printPoints();

    return 0;
}

Overwriting quad_tree.cpp


In [ ]:
!g++ -o quad_tree quad_tree.cpp
!./quad_tree

Node Boundary: (0, 0) - (100, 100)
Points stored in this node:
(10, 10) -z-value: 0
(20, 20) -z-value: 0
(30, 30) -z-value: 0
(74, 43) -z-value: 0

  Node Boundary: (0, 0) - (50, 50)
Points stored in this node:
(5, 5) -z-value: 1
(15, 15) -z-value: 1
(35, 35) -z-value: 1
(40, 6) -z-value: 1

  Node Boundary: (50, 0) - (100, 50)
Points stored in this node:
(54, 25) -z-value: 1

  Node Boundary: (0, 50) - (50, 100)
Points stored in this node:
(4, 70) -z-value: 1
(2, 80) -z-value: 1

  Node Boundary: (50, 50) - (100, 100)
Points stored in this node:
(90, 90) -z-value: 1
(60, 60) -z-value: 1
(70, 70) -z-value: 1
(80, 80) -z-value: 1



In [12]:
class KnowledgeBase:
    def __init__(self):
        self.assembly_relations = {}

    def add_assembly(self, part, subpart):
        if part not in self.assembly_relations:
            self.assembly_relations[part] = []
        self.assembly_relations[part].append(subpart)

    def query_components(self, part):
        components = set()
        self._query_components(part, components)
        return list(components)

    def _query_components(self, part, components):
        if part in self.assembly_relations:
            for subpart in self.assembly_relations[part]:
                components.add(subpart)
                self._query_components(subpart, components)

    def query_assembly(self, part, subpart):
        if part in self.assembly_relations:
            return subpart in self.assembly_relations[part]
        return False

    def query_assembled_from(self, subpart):
        assembled_parts = []
        for part, subparts in self.assembly_relations.items():
            if subpart in subparts:
                assembled_parts.append(part)
        return assembled_parts

def main():
    kb = KnowledgeBase()

    while True:
        print("1. Add assembly relations")
        print("2. Query components")
        print("3. Query assembly")
        print("4. Query assembled from")
        print("5. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            print("Enter assembly relations (one per line, format: part:subpart). Type 'done' to finish.")
            relations = []
            while True:
                line = input()
                if line.lower() == 'done':
                    break
                relations.append(line)
            for relation in relations:
                part, subpart = relation.split(':')
                kb.add_assembly(part, subpart)
        elif choice == "2":
            part = input("Enter part: ")
            components = kb.query_components(part)
            print("Components:", components)
        elif choice == "3":
            part = input("Enter part: ")
            subpart = input("Enter subpart: ")
            if kb.query_assembly(part, subpart):
                print("Yes, {} is a direct component of {}".format(subpart, part))
            else:
                print("No, {} is not a direct component of {}".format(subpart, part))
        elif choice == "4":
            subpart = input("Enter subpart: ")
            assembled_parts = kb.query_assembled_from(subpart)
            print("Assembled parts:", assembled_parts)
        elif choice == "5":
            break
        else:
            print("Invalid choice")

if __name__ == "__main__":
    main()

"""
tricycle:frame
tricycle:wheels
tricycle:handlebar
frame:pedal
frame:seat
frame:mudguard
wheels:front_wheel
wheels:rear_wheel
front_wheel:spokes
rear_wheel:spokes
handlebar:grips
handlebar:brake_lever
brake_lever:hydraulic_hose

Find all components of the tricycle:

tricycle
Find all subcomponents of the handlebar:

handlebar
Check if the hydraulic hose is a direct component of the tricycle:

tricycle:hydraulic_hose
Check if the seat is a direct component of the rear wheel:

rear_wheel:seat
Find all parts assembled from spokes:

spokes
Check if the seat is a component of any part in the tricycle:

tricycle
(then check if seat is in the output)

Find all parts that are directly assembled by the handlebar:

handlebar
"""

1. Add assembly relations
2. Query components
3. Query assembly
4. Query assembled from
5. Exit
Choose an option: 5


'\ntricycle:frame\ntricycle:wheels\ntricycle:handlebar\nframe:pedal\nframe:seat\nframe:mudguard\nwheels:front_wheel\nwheels:rear_wheel\nfront_wheel:spokes\nrear_wheel:spokes\nhandlebar:grips\nhandlebar:brake_lever\nbrake_lever:hydraulic_hose \n\nFind all components of the tricycle:\n\ntricycle\nFind all subcomponents of the handlebar:\n\nhandlebar\nCheck if the hydraulic hose is a direct component of the tricycle:\n\ntricycle:hydraulic_hose\nCheck if the seat is a direct component of the rear wheel:\n\nrear_wheel:seat\nFind all parts assembled from spokes:\n\nspokes\nCheck if the seat is a component of any part in the tricycle:\n\ntricycle\n(then check if seat is in the output)\n\nFind all parts that are directly assembled by the handlebar:\n\nhandlebar\n'

tricycle frame 1
tricycle wheels 1
tricycle handlebar 1
frame pedal 1
frame seat 1
`frame mudguard 1`
wheels front_wheel 1
wheels rear_wheel 1
front_wheel spokes 1
rear_wheel spokes 1
handlebar grips 1
handlebar brake_lever 1
brake_lever hydraulic_hose 1


In [ ]:
pip install lxml

In [ ]:
from lxml import etree

# Parse the XML file
tree = etree.parse("doc1.xml")

# Define a function to execute XPath queries
def execute_xpath(query):
    results = tree.xpath(query)
    for result in results:
        print(result.text)

# Example XPath queries
queries = [
    "/bookstore/book/title",           # Select titles of all books
    "/bookstore/book[price>35]",       # Select books with a price greater than 35
    "//book[year=2005]/title",         # Select titles of books published in 2005
    "//book[author='J K. Rowling']/title" # Select titles of books by J K. Rowling
]

# Execute the XPath queries
for query in queries:
    print("Executing XPath query:", query)
    execute_xpath(query)
    print()


Executing XPath query: /bookstore/book/title
Everyday Italian
Harry Potter
XQuery Kick Start
Learning XML- Erik T. Ray

Executing XPath query: /bookstore/book[price>35]

  

  

Executing XPath query: //book[year=2005]/title
Everyday Italian
Harry Potter

Executing XPath query: //book[author='J K. Rowling']/title
Harry Potter



In [28]:
import multiprocessing

def hash_function(key, table_size):
    """
    Simple hash function using the division method.

    Args:
    - key: The value to be hashed.
    - table_size: The size of the hash table.

    Returns:
    - The hash value.
    """
    return key % table_size

def build_hash_table(data, table_size):
    """
    Build a hash table from the given data.

    Args:
    - data: List of tuples (key, value).
    - table_size: The size of the hash table.

    Returns:
    - Dictionary representing the hash table.
    """
    hash_table = {}
    for key, value in data:
        hash_key = hash_function(key, table_size)
        if hash_key not in hash_table:
            hash_table[hash_key] = []
        hash_table[hash_key].append((key, value))
    return hash_table

def probe_partition(partition, hash_table, results):
    """
    Probe the hash table with the partition data.

    Args:
    - partition: List of tuples (key, value).
    - hash_table: Dictionary representing the hash table.
    - results: List to store the join results.
    """
    for key, value in partition:
        hash_key = hash_function(key, len(hash_table))
        if hash_key in hash_table:
            for entry in hash_table[hash_key]:
                if entry[0] == key:
                    results.append((key, value, entry[1]))

def parallel_hash_join(data_R, data_S, table_size):
    """
    Perform parallel hash join on datasets R and S.

    Args:
    - data_R: List of tuples (key, value) for dataset R.
    - data_S: List of tuples (key, value) for dataset S.
    - table_size: The size of the hash table.

    Returns:
    - List of tuples representing the join results.
    """
    hash_table_R = build_hash_table(data_R, table_size)
    hash_table_S = build_hash_table(data_S, table_size)

    manager = multiprocessing.Manager()
    results = manager.list()

    processes = []
    for partition_S in hash_table_S.values():
        process = multiprocessing.Process(target=probe_partition, args=(partition_S, hash_table_R, results))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    return results

if __name__ == "__main__":
    # Example datasets with more rows
    data_R = [(1, 'A'), (2, 'B'), (3, 'C'), (4, 'D'), (5, 'E')]
    data_S = [(1, 'X'), (2, 'Y'), (4, 'Z'), (6, 'W'), (7, 'Q')]

    # Example table size
    table_size = 5

    # Perform parallel hash join
    results = parallel_hash_join(data_R, data_S, table_size)

    # Print results
    print("Join Results:")
    for result in results:
        print(result)

Join Results:
(1, 'X', 'A')
(2, 'Y', 'B')
(4, 'Z', 'D')
